In [23]:
# dependencies
import boto3
import pandas as pd
from io import StringIO

## Dataframes to upload to AWS/postgres

In [24]:
# 1 (produced in: 'get_top40_audio_features.ipynb')
# this file has single instances of each song in the top40 dataset, including audio/genre data for each song
songs_audio = pd.read_csv('top40_audio_data_sample.csv')

# 2 (produced in: )
# this file shows all instances of each song in the dataset, but no audio/genre data
# we will have to join this df with the above df to produce a complete dataset, per the project rubric
charts_spine = pd.read_csv('top40_charts_spine.csv')

In [25]:
charts_spine.head()

,chart_order,week,position,song,artist,combo_str
0,0,1997-07-07,1,Mmm Bop,Hanson,HansonMmm Bop
1,1,1997-07-07,2,Bitch,Meredith Brooks,Meredith BrooksBitch
2,2,1997-07-07,3,Say You'll Be There,Spice Girls,Spice GirlsSay You'll Be There
3,3,1997-07-07,4,Return Of The Mack,Mark Morrison,Mark MorrisonReturn Of The Mack
4,4,1997-07-07,5,Sunny Came Home,Shawn Colvin,Shawn ColvinSunny Came Home


In [26]:
songs_audio.head()

,combo,week,position,artist,song,artist_id,song_id,acousticness,danceability,duration,...,virginia_hip_hop,vocal_house,vocal_jazz,washington_indie,west_coast_rap,world,worship,yacht_rock,yakut_pop,yodeling
0,0SdiiPkr02EUdekHZJkt580lnxrQAd9ZxbhBBe7d8FO8,7/7/97,1,Hanson,Mmm Bop,0SdiiPkr02EUdekHZJkt58,0lnxrQAd9ZxbhBBe7d8FO8,0.00481,0.683,268653,...,0,0,0,0,0,0,0,0,0,0
1,2QmLFuIDtNDmmJY3OtvinN3i6qNxyVgIdUZTTi5m25EM,7/7/97,2,Meredith Brooks,Bitch,2QmLFuIDtNDmmJY3OtvinN,3i6qNxyVgIdUZTTi5m25EM,0.01120,0.617,252760,...,0,0,0,0,0,0,0,0,0,0
2,0uq5PttqEjj3IH1bzwcrXF1yTQ39my3MoNROlFw3RDNy,7/7/97,3,Spice Girls,Say You'll Be There,0uq5PttqEjj3IH1bzwcrXF,1yTQ39my3MoNROlFw3RDNy,0.01490,0.726,235973,...,0,0,0,0,0,0,0,0,0,0
3,6V3F8MZrOKdT9fU686ybE93jDdpx9PMlfMBS5tOBHFm9,7/7/97,4,Mark Morrison,Return Of The Mack,6V3F8MZrOKdT9fU686ybE9,3jDdpx9PMlfMBS5tOBHFm9,0.00631,0.715,213093,...,0,0,0,0,0,0,0,0,0,0
4,0K7VN4aHxHcEb7PqkfoIVA4mOxpj82q6n3EO7HBZCelX,7/7/97,5,Shawn Colvin,Sunny Came Home,0K7VN4aHxHcEb7PqkfoIVA,4mOxpj82q6n3EO7HBZCelX,0.34200,0.558,264200,...,0,0,0,0,0,0,0,0,0,0


## Files for reference

In [27]:
# 3 (produced in: 'explore_top40_charts.ipynb')
# this file has all instances of each song, plus genre/audio data
# this was merged from files 1 + 2 in pandas, it is effectively the file we need to produce in SQL
charts_audio = pd.read_csv('top40_charts_audio.csv')

In [28]:
charts_audio.head()

,chart_order,week,position,song,artist,combo_str,combo,artist_id,song_id,acousticness,...,virginia_hip_hop,vocal_house,vocal_jazz,washington_indie,west_coast_rap,world,worship,yacht_rock,yakut_pop,yodeling
0,0,1997-07-07,1,Mmm Bop,Hanson,HansonMmm Bop,0SdiiPkr02EUdekHZJkt580lnxrQAd9ZxbhBBe7d8FO8,0SdiiPkr02EUdekHZJkt58,0lnxrQAd9ZxbhBBe7d8FO8,0.00481,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1997-07-07,2,Bitch,Meredith Brooks,Meredith BrooksBitch,2QmLFuIDtNDmmJY3OtvinN3i6qNxyVgIdUZTTi5m25EM,2QmLFuIDtNDmmJY3OtvinN,3i6qNxyVgIdUZTTi5m25EM,0.01120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1997-07-07,3,Say You'll Be There,Spice Girls,Spice GirlsSay You'll Be There,0uq5PttqEjj3IH1bzwcrXF1yTQ39my3MoNROlFw3RDNy,0uq5PttqEjj3IH1bzwcrXF,1yTQ39my3MoNROlFw3RDNy,0.01490,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,1997-07-07,4,Return Of The Mack,Mark Morrison,Mark MorrisonReturn Of The Mack,6V3F8MZrOKdT9fU686ybE93jDdpx9PMlfMBS5tOBHFm9,6V3F8MZrOKdT9fU686ybE9,3jDdpx9PMlfMBS5tOBHFm9,0.00631,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,1997-07-07,5,Sunny Came Home,Shawn Colvin,Shawn ColvinSunny Came Home,0K7VN4aHxHcEb7PqkfoIVA4mOxpj82q6n3EO7HBZCelX,0K7VN4aHxHcEb7PqkfoIVA,4mOxpj82q6n3EO7HBZCelX,0.34200,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Insert AWS-related code below

In [29]:
client = boto3.client('s3') # client variable 

## List Function
### The below allows you to call all files in the S3 without direct access.

In [30]:
# List Function from the S3

bucket_name = 'ucsd.final.project.music'
response = client.list_objects_v2(Bucket=bucket_name)
files = response.get("Contents")
for file in files:
    print(f"File_Name: {file['Key']}, Size: {file['Size']}, Date_M: {file['LastModified']}")

File_Name: top40_audio_data_sample.csv, Size: 2467382, Date_M: 2022-10-24 19:26:25+00:00
File_Name: top40_charts_audio.csv, Size: 62112155, Date_M: 2022-10-24 20:34:57+00:00
File_Name: top40_charts_spine.csv, Size: 2132646, Date_M: 2022-10-24 20:35:06+00:00
File_Name: top40_songs_audio.csv, Size: 2467382, Date_M: 2022-10-24 20:35:14+00:00
File_Name: top50.csv, Size: 3873, Date_M: 2022-10-16 22:51:49+00:00


## Write Function
### Repeat this function as necessary.

In [36]:
# Set df_to_upload to desired dataframe to Write
df_to_upload = charts_audio

In [41]:
df_to_upload.head(3)

,chart_order,week,position,song,artist,combo_str,combo,artist_id,song_id,acousticness,...,virginia_hip_hop,vocal_house,vocal_jazz,washington_indie,west_coast_rap,world,worship,yacht_rock,yakut_pop,yodeling
0,0,1997-07-07,1,Mmm Bop,Hanson,HansonMmm Bop,0SdiiPkr02EUdekHZJkt580lnxrQAd9ZxbhBBe7d8FO8,0SdiiPkr02EUdekHZJkt58,0lnxrQAd9ZxbhBBe7d8FO8,0.00481,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1997-07-07,2,Bitch,Meredith Brooks,Meredith BrooksBitch,2QmLFuIDtNDmmJY3OtvinN3i6qNxyVgIdUZTTi5m25EM,2QmLFuIDtNDmmJY3OtvinN,3i6qNxyVgIdUZTTi5m25EM,0.01120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1997-07-07,3,Say You'll Be There,Spice Girls,Spice GirlsSay You'll Be There,0uq5PttqEjj3IH1bzwcrXF1yTQ39my3MoNROlFw3RDNy,0uq5PttqEjj3IH1bzwcrXF,1yTQ39my3MoNROlFw3RDNy,0.01490,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
to_s3_upload_name = input('What do you want to name this file in the S3 Bucket?  Do not forget the file extension: ')

What do you want to name this file in the S3 Bucket?  Do not forget the file extension: charts_audio_test.csv


In [43]:
to_s3_upload_name

'charts_audio_test.csv'

In [44]:
# If HTTPStatusCode = 200 and the RetryAttempts = 0 this is a good indicator of success.
filename = to_s3_upload_name
bucketName = 'ucsd.final.project.music'

csv_buffer = StringIO()
df_to_upload.to_csv(csv_buffer)

response = client.put_object(
    ACL='private',
    Body=csv_buffer.getvalue(),
    Bucket=bucketName,
    Key=filename)

response

{'ResponseMetadata': {'RequestId': 'AZZVJQA54X5B7XTW',
  'HostId': 'Y0nurDWS/5TljBcs/gu+JGAf7mW4cJJuHsb4p0/irE0y9ZmlDZaisbAsdgnS11Ciye9V8D7IT/M=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Y0nurDWS/5TljBcs/gu+JGAf7mW4cJJuHsb4p0/irE0y9ZmlDZaisbAsdgnS11Ciye9V8D7IT/M=',
   'x-amz-request-id': 'AZZVJQA54X5B7XTW',
   'date': 'Mon, 24 Oct 2022 21:28:50 GMT',
   'etag': '"6fd2b907e78a1c94a8c1f7baded4e175"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"6fd2b907e78a1c94a8c1f7baded4e175"'}

## Read Function

In [46]:
file_name = input("What file do you want to pull into a dataframe? Don't forget the extension: ")
file_name

What file do you want to pull into a dataframe? Don't forget the extension: top50.csv


'top50.csv'

In [47]:
path = 's3://ucsd.final.project.music/' +str(file_name)
path

's3://ucsd.final.project.music/top50.csv'

In [48]:
df = pd.read_csv(path)

In [49]:
df.head()

,Unnamed: 0,Track.Name,Artist.Name,Genre,Beats.Per.Minute,Energy,Danceability,Loudness..dB..,Liveness,Valence.,Length.,Acousticness..,Speechiness.,Popularity
0,1,Señorita,Shawn Mendes,canadian pop,117,55,76,-6,8,75,191,4,3,79
1,2,China,Anuel AA,reggaeton flow,105,81,79,-4,8,61,302,8,9,92
2,3,boyfriend (with Social House),Ariana Grande,dance pop,190,80,40,-4,16,70,186,12,46,85
3,4,Beautiful People (feat. Khalid),Ed Sheeran,pop,93,65,64,-8,8,55,198,12,19,86
4,5,Goodbyes (Feat. Young Thug),Post Malone,dfw rap,150,65,58,-4,11,18,175,45,7,94
